In [1]:
import torch
import numpy as np
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

RuntimeError: Failed to import transformers.models.bert.modeling_bert because of the following error (look up to see its traceback):
cannot import name 'PROTOCOL_TLS' from 'urllib3.util.ssl_' (/home/liu/anaconda3/lib/python3.9/site-packages/urllib3/util/ssl_.py)

In [ ]:
# !pip install paddlenlp
# Check if GPU is available and set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

In [ ]:
### read data
qwen = pd.read_csv("../data/qwen_v5.csv")
ernie = pd.read_csv("../data/ernie_v5.csv")
baidu = pd.read_csv("../data/baidu_v4.csv")

In [ ]:
model_name = 'bert-base-chinese'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name).to(device)


In [ ]:
def get_bert_embedding(text):
    # Handle potential non-string elements by converting to string
    if not isinstance(text, str):
        text = str(text)
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
    embedding = torch.mean(outputs.last_hidden_state, dim=1).cpu().numpy()
    return embedding

ernie["word_embedding_json_response"] = ernie["json_response"].apply(get_bert_embedding)
qwen["word_embedding_json_response"] = qwen["json_response"].apply(get_bert_embedding)
baidu["word_embedding_response_values"] = baidu["response_values"].apply(get_bert_embedding)

In [ ]:
ernie.to_csv("ernie_v4.csv",index = False,encoding='utf-8')
qwen.to_csv("qwen_v4.csv",index = False,encoding='utf-8')
baidu.to_csv("baidu_v4.csv",index= False,encoding='utf-8')